In [ ]:
pip install pytesseract pdf2image pillow
pip install langchain tiktoken

In [17]:
# Get Khmer OCR model
import pytesseract

# Point to the correct path where tesseract.exe is installed
pytesseract.pytesseract.tesseract_cmd = r'/Users/cheamenghour/Documents/Cambodia Agriculture/tesseract-OCR/tesseract.exe'

# Print available OCR languages
print(pytesseract.get_languages(config=''))


['eng', 'khm', 'osd']


In [32]:
import os
from pdf2image import convert_from_path

folder_dir = '/Users/cheamenghour/Documents/Cambodia Agriculture/ស្រូវ'
poppler_path = r'/Users/cheamenghour/Documents/Cambodia Agriculture/Release-24.08.0-0/poppler-24.08.0/Library/bin'


for root, dirs, files in os.walk(folder_dir):
    for file in files:
        if file.lower().endswith('.pdf'):
            file_path = os.path.join(root, file)
            images = convert_from_path(file_path, poppler_path = poppler_path)
            text_folder = os.path.join(root, 'text')
            os.makedirs(text_folder, exist_ok = True)
            for i, img in enumerate(images):
                text = pytesseract.image_to_string(img, lang ='khm')
                file = os.path.splitext(file)[0]  # removes .pdf
                text_path = os.path.join(text_folder, f'{file}_page_{i+1}.text')
                with open(text_path, 'w', encoding = 'utf-8') as f:
                    f.write(text)


In [10]:
print(len(images))

2


In [6]:
# Table detection

import pdfplumber
from openpyxl import Workbook
import os

# Set your input and output paths
pdf_path = "/Users/cheamenghour/Documents/Cambodia Agriculture/ស្រូវ/១២ ជំហានដើម្បីផលិតស្រូវបានជោគជ័យ/១២ ជំហានដើម្បីផលិតស្រូវបានជោគជ័យ.pdf"
xlsx_output = "/Users/cheamenghour/Documents/Cambodia Agriculture/ស្រូវ/១២ ជំហានដើម្បីផលិតស្រូវបានជោគជ័យ/table/table_output.xlsx"

# Create a new Excel workbook and sheet
wb = Workbook()
ws = wb.active
ws.title = "Extracted Tables"

# Open PDF and extract tables
with pdfplumber.open(pdf_path) as pdf:
    for page_num, page in enumerate(pdf.pages, start=1):
        tables = page.extract_tables()
        for table in tables:
            # Add a page header before each table (optional)
            ws.append([f"--- Table from Page {page_num} ---"])
            for row in table:
                clean_row = [cell.replace("\n", " ").strip() if cell else "" for cell in row]
                ws.append(clean_row)
            ws.append([])  # Add a blank row between tables

# Save the file (no need to worry about encoding for .xlsx)
wb.save(xlsx_output)

print(f"✅ Table data extracted and saved to:\n{xlsx_output}")



✅ Table data extracted and saved to:
/Users/cheamenghour/Documents/Cambodia Agriculture/ស្រូវ/១២ ជំហានដើម្បីផលិតស្រូវបានជោគជ័យ/table/table_output.xlsx


In [2]:
# Table to text and convert to chunk text.

import pandas as pd

def table_to_text(path):
    df = pd.read_excel(path)
    text_chunks = []

    for index, row in df.iterrows():
        row_text = " | ".join([f"{col}: {row[col]}" for col in df.columns])
        text_chunks.append(row_text)

    return "\n".join(text_chunks)

# Example usage:
table_text = table_to_text("D:/Cambodia Agriculture/Text data/ពូជស្រូវ ដំណើបស្បៃមង្គល/table/table_2.xlsx")

# Optional: Save it or include in your main chunk
with open("D:/Cambodia Agriculture/Text data/ពូជស្រូវ ដំណើបស្បៃមង្គល/chunk text/table2_chunk.txt", "w", encoding="utf-8") as f:
    f.write("--- chunk ---\n")
    f.write(table_text)